In [1]:
import numpy as np
import pandas as pd 
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler ,PolynomialFeatures
import seaborn as sns
import time
from sklearn.svm import SVR
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split ,cross_val_score ,StratifiedKFold ,KFold
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.linear_model import ElasticNet, BayesianRidge, HuberRegressor, Lars
import itertools
from dateutil.relativedelta import relativedelta
from datetime import datetime
import threading
import time 
import multiprocessing
from multiprocessing import Pool
import concurrent.futures
from functools import partial
from multiprocessing import Process, Manager

In [2]:

inputs_df = pd.read_csv(r"C:\Users\97155\Downloads\inputs_df.csv"  )
inputs_df.set_index('CURVE_START_DT', inplace=True)
x_parameters=['NEWC Fwd', 'JKM', 'ICE Brent Fwd (3-0-1)', 'GDP', 'Nuclear', 'Solar', 'HDDs', 'CDDs']
y_parameter = 'power_mmcm'
regression_start_date = '2019-01-01'
regression_end_date = '2020-02-01'
regression_degrees = 2
num_duplicates = 2
seed = 0

In [3]:
regression_df = inputs_df[(inputs_df.index >= regression_start_date) & (inputs_df.index <= regression_end_date)].copy()
y_train = regression_df[y_parameter].values.reshape(-1,1)
X_train = regression_df[x_parameters]
y_train_duplicate = pd.concat([pd.DataFrame(y_train)] * num_duplicates, ignore_index=True,axis=0)
X_train_duplicate = pd.concat([pd.DataFrame(X_train)] * num_duplicates, ignore_index=True,axis=0)

In [4]:
forecast_df = inputs_df[x_parameters + [y_parameter]].dropna(subset=x_parameters, how='any')
out_sample = forecast_df[(forecast_df.index < regression_start_date) | (forecast_df.index > regression_end_date)].dropna(how='any')
y_test = out_sample[y_parameter].values.reshape(-1,1)
X_test = out_sample[x_parameters]

In [5]:
poly_reg = PolynomialFeatures(degree=regression_degrees, include_bias=False)
scl = MinMaxScaler(feature_range=(0, 1))

poly_features_train = poly_reg.fit_transform(X_train)
scal_features_train = scl.fit_transform(X_train)
scal_poly_features_train = poly_reg.fit_transform(scal_features_train)

poly_features_test = poly_reg.fit_transform(X_test)
scal_features_test = scl.fit_transform(X_test)
scal_poly_features_test = poly_reg.fit_transform(scal_features_test)

poly_features_train_duplicate = poly_reg.fit_transform(X_train_duplicate)
scal_features_train_duplicate = scl.fit_transform(X_train_duplicate)
scal_poly_features_train_duplicate = poly_reg.fit_transform(scal_features_train_duplicate)

In [6]:
svr_model = SVR()
rf_regressor = RandomForestRegressor(n_estimators=450, random_state=seed)
lin_reg_model = LinearRegression()
alpha = 0.1  # L1 regularization strength
l1_ratio = 0.5  # Mixing parameter (0.5 for an equal mix of L1 and L2)
elnt_model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
bays_model = BayesianRidge()
Hur_model = HuberRegressor(epsilon=1.35)
Lars_model = Lars()

In [7]:

result_lock = threading.Lock()

def GitScore(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train)
    score = model.score(x_test, y_test)
    return score

In [8]:
def get_reg_unique_dates(start_date, end_date):
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    dates_delta = relativedelta(end_date, start_date)
    num_months = dates_delta.months + (dates_delta.years * 12) + 1
    dates_list = []
    for i in range(num_months):
        d1 = start_date + relativedelta(months=i)
        for k in range(i, num_months):
            d2 = start_date + relativedelta(months=k)            
            dates_list.append([d1, d2])
    return dates_list

start_date = '2018-01-01'
end_date= '2023-04-01'

dates_list= get_reg_unique_dates(start_date, end_date)


filtered_dates_list = []
for i in dates_list:
    if datetime.strptime(str(i[0]),"%Y-%m-%d %H:%M:%S").year != datetime.strptime(str(i[1]),"%Y-%m-%d %H:%M:%S").year:
        filtered_dates_list.append(i)


In [9]:
models=[rf_regressor,lin_reg_model,svr_model ,elnt_model, bays_model]

xs = ['HDDs', 'CDDs', 'NEWC Fwd', 'JKM', 'ICE Brent Fwd (3-0-1)', 'GDP', 'Nuclear', 'Solar']
xs_list = []
num_xs = len(xs)

for i in range(len(xs) + 1):
    for subset in itertools.combinations(xs, i):
        # print(subset)
        #if len(subset) > 0:
        xs_list.append(subset)

In [10]:
def split_list(input_list, n):
    # Calculate the number of elements in each sublist
    sublist_size = len(input_list) // n
    remainder = len(input_list) % n

    # Initialize variables
    sublists = []
    start = 0

    for i in range(n):
        # Calculate the end index for the current sublist
        end = start + sublist_size + (1 if i < remainder else 0)

        # Append the sublist to the result
        sublists.append(input_list[start:end])

        # Update the start index for the next sublist
        start = end

    return sublists

In [11]:
# def data_preparing(filtered_dates_list_1, shared_dict, time_range, lock):

#     for i in filtered_dates_list_1:
#         regression_start_date = i[0]
#         regression_end_date = i[1]
#         regression_df = inputs_df[(inputs_df.index >= str(regression_start_date)) & (inputs_df.index <= str(regression_end_date))].dropna(how='any').copy()
#         y_train = regression_df[y_parameter].values.reshape(-1,1)
#         X_train = regression_df[x_parameters]
    
#         forecast_df = inputs_df[x_parameters + [y_parameter]].dropna(subset=x_parameters, how='any')
#         out_sample = forecast_df[(forecast_df.index < str(regression_start_date)) | (forecast_df.index > str(regression_end_date))].dropna(how='any')
#         y_test = out_sample[y_parameter].values.reshape(-1,1)
#         X_test = out_sample[x_parameters]
#         y_train_duplicate = pd.concat([pd.DataFrame(y_train)] * num_duplicates, ignore_index=True,axis=0)
#         X_train_duplicate = pd.concat([pd.DataFrame(X_train)] * num_duplicates, ignore_index=True,axis=0)

#         poly_reg = PolynomialFeatures(degree=regression_degrees, include_bias=False)
#         scl = MinMaxScaler(feature_range=(0, 1))
#         changing = {}
#         if X_train.shape[0] !=0:
#             poly_features_train = poly_reg.fit_transform(X_train)
#             scal_features_train = scl.fit_transform(X_train)
#             scal_poly_features_train = poly_reg.fit_transform(scal_features_train)
        
#             poly_features_test = poly_reg.fit_transform(X_test)
#             scal_features_test = scl.fit_transform(X_test)
#             scal_poly_features_test = poly_reg.fit_transform(scal_features_test)
        
#             poly_features_train_duplicate = poly_reg.fit_transform(X_train_duplicate)
#             scal_features_train_duplicate = scl.fit_transform(X_train_duplicate)
#             scal_poly_features_train_duplicate = poly_reg.fit_transform(scal_features_train_duplicate)
                    
#             x_train_types = {'X_train': X_train, 
#                              'scal_features_train': scal_features_train,
#                              'poly_features_train': poly_features_train,
#                              'scal_poly_features_train': scal_poly_features_train}
            
#             x_test_types = [X_test, scal_features_test, poly_features_test, scal_poly_features_test]
            
#             x_train_dupl_types = {'X_train_duplicate': X_train_duplicate, 
#                                   'scal_features_train_duplicate': scal_features_train_duplicate,
#                                   'poly_features_train_duplicate': poly_features_train_duplicate,
#                                   'scal_poly_features_train_duplicate': scal_poly_features_train_duplicate}
            
#             y_train_types = [y_train, y_train_duplicate]
#             for i in models:
#                 scores = {}
#                 for j in range(len(x_test_types)):
#                     lock.acquire()
#                     s_x= {}
#                     s_x_dupl= {}
#                     score_x = GitScore(i, list(x_train_types.values())[j], y_train, x_test_types[j], y_test)
#                     score_x_dupl = GitScore(i, list(x_train_dupl_types.values())[j], y_train_duplicate, x_test_types[j], y_test)
#                     s_x[f'{list(x_train_types.keys())[j]}'] = score_x
#                     s_x_dupl[f'{list(x_train_dupl_types.keys())[j]}'] =  score_x_dupl
                    
#                     scores.update(s_x)
#                     scores.update(s_x_dupl)
#                     lock.release()
#                 changing[f'{i}'] = scores
#             time_range[f'{regression_start_date} _ {regression_end_date}']=changing
#         else:
#             pass
#         shared_dict[str(x_parameters)]=time_range
#         with lock:
#             return shared_dict
       
            
            
# if __name__ == '__main__':
#     xs_list_1= xs_list[37:]
#     xs_list_2 = xs_list_1[0:1]
#     for x_par in xs_list_2:
#         lock = multiprocessing.Lock()
#         filtered_dates_list_1 = filtered_dates_list[0:20]
#         x_parameters = list(x_par)
#         time_range = {}

#         filtered_dates_list_splitted = split_list(filtered_dates_list_1, 2)
#         threads = []

#         for dates_list in filtered_dates_list_splitted:
#             manager = multiprocessing.Manager()
#             shared_dict = manager.dict()
#             time_range = manager.dict()
#             thread = multiprocessing.Process(target=data_preparing, args=(dates_list, shared_dict, time_range,lock))
#             threads.append(thread)
#             thread.start()
     
#         for thread in threads:
#             thread.join()

In [12]:
# print(shared_dict)

In [13]:
# def data_preparing(filtered_dates_list_1):
#     lock = multiprocessing.Lock()
#     for i in filtered_dates_list_1:
#         regression_start_date = i[0]
#         regression_end_date = i[1]
#         regression_df = inputs_df[(inputs_df.index >= str(regression_start_date)) & (inputs_df.index <= str(regression_end_date))].dropna(how='any').copy()
#         y_train = regression_df[y_parameter].values.reshape(-1,1)
#         X_train = regression_df[x_parameters]
    
#         forecast_df = inputs_df[x_parameters + [y_parameter]].dropna(subset=x_parameters, how='any')
#         out_sample = forecast_df[(forecast_df.index < str(regression_start_date)) | (forecast_df.index > str(regression_end_date))].dropna(how='any')
#         y_test = out_sample[y_parameter].values.reshape(-1,1)
#         X_test = out_sample[x_parameters]
#         y_train_duplicate = pd.concat([pd.DataFrame(y_train)] * num_duplicates, ignore_index=True,axis=0)
#         X_train_duplicate = pd.concat([pd.DataFrame(X_train)] * num_duplicates, ignore_index=True,axis=0)

#         poly_reg = PolynomialFeatures(degree=regression_degrees, include_bias=False)
#         scl = MinMaxScaler(feature_range=(0, 1))
#         changing = {}
#         if X_train.shape[0] !=0:
#             poly_features_train = poly_reg.fit_transform(X_train)
#             scal_features_train = scl.fit_transform(X_train)
#             scal_poly_features_train = poly_reg.fit_transform(scal_features_train)
        
#             poly_features_test = poly_reg.fit_transform(X_test)
#             scal_features_test = scl.fit_transform(X_test)
#             scal_poly_features_test = poly_reg.fit_transform(scal_features_test)
        
#             poly_features_train_duplicate = poly_reg.fit_transform(X_train_duplicate)
#             scal_features_train_duplicate = scl.fit_transform(X_train_duplicate)
#             scal_poly_features_train_duplicate = poly_reg.fit_transform(scal_features_train_duplicate)
                    
#             x_train_types = {'X_train': X_train, 
#                              'scal_features_train': scal_features_train,
#                              'poly_features_train': poly_features_train,
#                              'scal_poly_features_train': scal_poly_features_train}
            
#             x_test_types = [X_test, scal_features_test, poly_features_test, scal_poly_features_test]
            
#             x_train_dupl_types = {'X_train_duplicate': X_train_duplicate, 
#                                   'scal_features_train_duplicate': scal_features_train_duplicate,
#                                   'poly_features_train_duplicate': poly_features_train_duplicate,
#                                   'scal_poly_features_train_duplicate': scal_poly_features_train_duplicate}
            
#             y_train_types = [y_train, y_train_duplicate]
#             for i in models:
#                 scores = {}
#                 for j in range(len(x_test_types)):
#                     lock.acquire()
#                     s_x= {}
#                     s_x_dupl= {}
#                     score_x = GitScore(i, list(x_train_types.values())[j], y_train, x_test_types[j], y_test)
#                     score_x_dupl = GitScore(i, list(x_train_dupl_types.values())[j], y_train_duplicate, x_test_types[j], y_test)
#                     s_x[f'{list(x_train_types.keys())[j]}'] = score_x
#                     s_x_dupl[f'{list(x_train_dupl_types.keys())[j]}'] =  score_x_dupl
                    
#                     scores.update(s_x)
#                     scores.update(s_x_dupl)
#                     lock.release()
#                 changing[f'{i}'] = scores
#             time_range[f'{regression_start_date} _ {regression_end_date}']=changing
#         else:
#             pass
#         with lock:
#             shared_dict[str(x_parameters)]=time_range
#         return shared_dict
       
# if __name__ == '__main__':
#     xs_list_1 = xs_list[37:]
#     xs_list_2 = xs_list_1[0:1]
#     filtered_dates_list_1 = filtered_dates_list[0:20]

#     p = Pool()
#     results = p.map(data_preparing, filtered_dates_list_1)


In [14]:
# print(thread)

In [ ]:
lock = multiprocessing.Lock()
time_range={}
fearure_secl= {}
def data_preparing(filtered_dates_list_1, x_parameters ,shared_dict, lock):
    
    for i in filtered_dates_list_1:
        regression_start_date = i[0]
        regression_end_date = i[1]
        regression_df = inputs_df[(inputs_df.index >= str(regression_start_date)) & (inputs_df.index <= str(regression_end_date))].dropna(how='any').copy()
        y_train = regression_df[y_parameter].values.reshape(-1,1)
        X_train = regression_df[x_parameters]
    
        forecast_df = inputs_df[x_parameters + [y_parameter]].dropna(subset=x_parameters, how='any')
        out_sample = forecast_df[(forecast_df.index < str(regression_start_date)) | (forecast_df.index > str(regression_end_date))].dropna(how='any')
        y_test = out_sample[y_parameter].values.reshape(-1,1)
        X_test = out_sample[x_parameters]
        y_train_duplicate = pd.concat([pd.DataFrame(y_train)] * num_duplicates, ignore_index=True,axis=0)
        X_train_duplicate = pd.concat([pd.DataFrame(X_train)] * num_duplicates, ignore_index=True,axis=0)
  
        poly_reg = PolynomialFeatures(degree=regression_degrees, include_bias=False)
        scl = MinMaxScaler(feature_range=(0, 1))
        changing = {}
        if X_train.shape[0] !=0:
            poly_features_train = poly_reg.fit_transform(X_train)
            scal_features_train = scl.fit_transform(X_train)
            scal_poly_features_train = poly_reg.fit_transform(scal_features_train)
        
            poly_features_test = poly_reg.fit_transform(X_test)
            scal_features_test = scl.fit_transform(X_test)
            scal_poly_features_test = poly_reg.fit_transform(scal_features_test)
        
            poly_features_train_duplicate = poly_reg.fit_transform(X_train_duplicate)
            scal_features_train_duplicate = scl.fit_transform(X_train_duplicate)
            scal_poly_features_train_duplicate = poly_reg.fit_transform(scal_features_train_duplicate)
                    
            x_train_types = {'X_train': X_train, 
                             'scal_features_train': scal_features_train,
                             'poly_features_train': poly_features_train,
                             'scal_poly_features_train': scal_poly_features_train}
            
            x_test_types = [X_test, scal_features_test, poly_features_test, scal_poly_features_test]
            
            x_train_dupl_types = {'X_train_duplicate': X_train_duplicate, 
                                  'scal_features_train_duplicate': scal_features_train_duplicate,
                                  'poly_features_train_duplicate': poly_features_train_duplicate,
                                  'scal_poly_features_train_duplicate': scal_poly_features_train_duplicate}
            
            y_train_types = [y_train, y_train_duplicate]
            for i in models:
                scores = {}
                for j in range(len(x_test_types)):
                    lock.acquire()
                    s_x= {}
                    s_x_dupl= {}
                    score_x = GitScore(i, list(x_train_types.values())[j], y_train, x_test_types[j], y_test)
                    score_x_dupl = GitScore(i, list(x_train_dupl_types.values())[j], y_train_duplicate, x_test_types[j], y_test)
                    s_x[f'{list(x_train_types.keys())[j]}'] = score_x
                    s_x_dupl[f'{list(x_train_dupl_types.keys())[j]}'] =  score_x_dupl
                    
                    scores.update(s_x)
                    scores.update(s_x_dupl)
                    lock.release()
                changing[f'{i}'] = scores
            time_range[f'{regression_start_date} _ {regression_end_date}']=changing
        else:
            pass
        fearure_secl[str(x_parameters)]=time_range
        with lock:
            shared_dict=fearure_secl
            
def combinations_preparing(x_parameters, filtered_dates_list):
    manager = multiprocessing.Manager()
    shared_dict = manager.dict()
    lock = manager.Lock()
    
    threads = []

    for dates_list in filtered_dates_list:
        thread = multiprocessing.Process(target=data_preparing, args=(dates_list, x_parameters, shared_dict, lock))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()

    return shared_dict

if __name__ == '__main__':
    xs_list_1 = xs_list[37:]
    xs_list_2 = xs_list_1[0:10]
    filtered_dates_list_1 = filtered_dates_list[0:20]

    p = Pool()
    results = p.starmap(combinations_preparing, [(x_par, filtered_dates_list_1) for x_par in xs_list_2])
